In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pkg_resources import resource_filename

from climpyrical.gridding import scale_model_obs
from climpyrical.data import read_data
from climpyrical.cmd.find_matched_model_vals import add_model_values

In [6]:
nrc_path = '/home/nannau/nrc_data/NBCC_2020_new_coords.xlsm'
df_nrc = pd.read_excel(nrc_path).iloc[:-1]

# fill problem values with better values from 2015
id_typo = df_nrc[(df_nrc['2020 Longitude'] > 0) | (df_nrc['2020 Latitude'] < 40)].index
df_nrc.loc[id_typo, '2020 Longitude'] = df_nrc['2015 Long.'].values[id_typo]
df_nrc.loc[id_typo, '2020 Latitude'] = df_nrc['2015 Lat.'].values[id_typo]

# process to get the model values
df_nrc.rename(columns={"2020 Longitude": "lon",
                      "2020 Latitude": "lat"})

,LocationID,SortOrder,Location,Prov,2015 Elev (m),2020 Elev (m),Was Elev. Changed,2015 Lat.,2015 Long.,lat,lon,Was Lat. Changed,Was Long. Changed
0,1.0,10.0,100 Mile House,BC,1040,926,Yes,51.6500,-121.2833,51.64,-121.3,Yes,Yes
1,2.0,20.0,Abbotsford,BC,70,70,No,49.0517,-122.3289,49.05,-122.33,Yes,Yes
2,3.0,30.0,Agassiz,BC,15,25,Yes,49.2300,-121.7700,49.24,-121.76,Yes,Yes
3,4.0,40.0,Alberni,BC,12,50,Yes,49.2700,-124.8000,49.27,-124.8,No,No
4,5.0,50.0,Ashcroft,BC,305,305,No,50.7200,-121.2800,50.72,-121.28,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,632.0,6760.0,Kugluktuk,NU,10,15,Yes,67.8300,-115.0800,67.83,-115.1,No,Yes
676,637.0,6770.0,Nottingham Island,NU,30,30,No,63.1000,-78.0000,63.12,-77.93,Yes,Yes
677,639.0,6780.0,Resolute,NU,25,20,Yes,74.6800,-94.9000,74.70,-94.83,Yes,Yes
678,640.0,6790.0,Resolution Island,NU,5,0,Yes,61.3000,-64.8833,61.31,-64.87,Yes,Yes


In [ ]:
# find indices of the station locations
ix, iy = find_element_wise_nearest_pos(
            ds.rlon.values, 
            ds.rlat.values, 
            df_nrc.rlon.values, 
            df_nrc.rlat.values
)
# write to new dataframe
df_nrc_matched = pd.DataFrame(
    {'ilocy': iy, 
     'ilocx': ix, 
     'Location': df_nrc.Location, 
     'Prov': df_nrc.Prov,
     'rlat':df_nrc.rlat, 
     'rlon':df_nrc.rlon, 
     'lon': df_nrc['2020 Longitude'], 
     'lat': df_nrc['2020 Latitude']
     }
)

df_nrc_matched.to_csv('./data/processed/nrc_locations.csv', index=False)

In [ ]:
matched_df_nrc = pd.read_csv('./data/processed/nrc_locations.csv')

nrc_nan_df = matched_df_nrc[np.isnan(ratio_final[matched_df_nrc.ilocy, matched_df_nrc.ilocx])]

ratio_nan = ~np.isnan(ratio_final)
    
f = NearestNDInterpolator(np.stack([rlon[ratio_nan], rlat[ratio_nan]]).T, ratio_final[ratio_nan])

table_nan = f(
    np.stack(
        [rlon[nrc_nan_df.ilocy.values, nrc_nan_df.ilocx.values], 
         rlat[nrc_nan_df.ilocy.values, nrc_nan_df.ilocx.values]
        ]).T).T

design_value = ratio_final[matched_df_nrc.ilocy, matched_df_nrc.ilocx]
dvnan = np.isnan(design_value)
design_value[dvnan] = table_nan

nrc_table = pd.DataFrame()
nrc_table[station_dv] = design_value
nrc_table = nrc_table.assign(
    Prov=matched_df_nrc.Prov,
    Location=matched_df_nrc.Location, 
    lat=matched_df_nrc.lat, 
    lon=matched_df_nrc.lon,
    rlat=matched_df_nrc.rlat, 
    rlon=matched_df_nrc.rlon)

assert not df.isnull().values.any()

nrc_table.round(3).to_csv("./data/reporting/"+file_dv+"_TableC2.csv", index=False)
nrc_table.round(3).to_csv("~/dash-dv-explorer/assets/data/tables/"+file_dv+"_TableC2.csv", index=False)